In [1]:
%matplotlib inline
import numpy as np

from transform import combine_tsv_files, load_process_and_store_spreadsheets

In [2]:
def harmonic_mean(t0, t1):
    return t0 * t1 / (t0 + t1)

load_process_and_store_spreadsheets(
    transformation=harmonic_mean,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

AttributeError: 'int' object has no attribute 'split'

In [4]:
def difference(t0, t1):
    return t1 - t0

load_process_and_store_spreadsheets(
    transformation=difference,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

In [5]:
def relative_difference(t0, t1):
    return (t1-t0)/t0

load_process_and_store_spreadsheets(
    transformation=relative_difference,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

<ipython-input-5-8639749480b7>:2: RuntimeWarning: divide by zero encountered in double_scalars
  return (t1-t0)/t0


In [6]:
def up_or_down(t0, t1):
    return np.sign(t1 - t0)


load_process_and_store_spreadsheets(
    transformation=up_or_down,
    all_filename_prefix="output/all_",
    train_filename_prefix="output/train_",
    test_filename_prefix="output/test_",
)

Data without train/test split, with CNV scores merged in.

In [7]:
def to_excel(snv_name, cnv_name):
    excel_filename = snv_name.replace('.tsv', '_CNV.xlsx')
    X, y = combine_tsv_files(snv_name, cnv_name)
    X.merge(y, left_index=True, right_index=True).to_excel(excel_filename)
    print(f'Wrote {excel_filename}.')

In [10]:
for coarseness in ('gene', 'pathway'):
    for transformation in ('harmonic_mean', 'difference', 'relative_difference'):
        to_excel(
            f"output/all__{coarseness}__{transformation}__Allele Fraction.tsv",
            f"output/all__{coarseness}__{transformation}__CNV Score.tsv",
        )
        to_excel(
            f"output/all__{coarseness}__{transformation}__No. Mutant Molecules per mL.tsv",
            f"output/all__{coarseness}__{transformation}__CNV Score.tsv",
        )

Wrote output/all__gene__harmonic_mean__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__harmonic_mean__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__gene__difference__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__difference__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__gene__relative_difference__Allele Fraction_CNV.xlsx.
Wrote output/all__gene__relative_difference__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__pathway__harmonic_mean__Allele Fraction_CNV.xlsx.
Wrote output/all__pathway__harmonic_mean__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__pathway__difference__Allele Fraction_CNV.xlsx.
Wrote output/all__pathway__difference__No. Mutant Molecules per mL_CNV.xlsx.
Wrote output/all__pathway__relative_difference__Allele Fraction_CNV.xlsx.
Wrote output/all__pathway__relative_difference__No. Mutant Molecules per mL_CNV.xlsx.
